In [ ]:
# Aula Prática nº4 - Agrupamento de Dados
# Grupo: Fabrício Fernandes Ziliotti - 11711BCC002
#	       Marcos Victor de Aquino Barra - 11711BCC007
#	       Salomão Oliveira Alves - 11711BCC038

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
#CELULA 1
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
import random
from sklearn.cluster import KMeans


In [ ]:
# CELULA 2

# ler o arquivo csv
def le_arq(arq_url):
  data = pd.read_csv(arq_url, header=None)
  classe = np.array(data.iloc[:,4])
  data = np.array(data.iloc[:,0:4])
  return data, classe


def exportToCsv(np_array, export_file_path, sswc, name):
  np_array.to_csv(export_file_path +"kmeans-"+ name + "-clusters.csv")
  arquivo = open(export_file_path +"silhueta_"+ name + "-clusters.txt", 'w')
  arquivo.write("Para o KMeans com " + name + "clusters, a silhueta simplificada é: " + sswc)
  arquivo.close()



# calcula a distância euclidiana entre dois pontos
def euclidiana(vetor1, vetor2):
	
	if(len(vetor1) != len(vetor2)):
		print(len(vetor1))
		print(len(vetor2))
	
		print("Os dois vetores devem possuir a mesma quantidade de elementos.")
		input()
		return

	qtdElementos, soma = len(vetor1), 0
	
	for i in range(qtdElementos):
		soma += math.pow(vetor1[i] - vetor2[i], 2)
  
	return math.sqrt(soma)
 

#Algoritmo K-Means
def Kcluster(data, k=3):
  
  #Escolhe aleatoriamente K centróides
  centroids_indexes = random.sample(range(len(data)), k=k)
  centroids = data[[centroids_indexes]]


  #O número de iterações será no máximo 100
  for t in range(100):
    #Cria uma lista contendo 3 lista vazias (vai ter id dentro dele). Esses serão os nossos clusters
    clusters = [[] for i in range(k)]
    
    #Verifica qual centroide esta mais perto de cada instância e insere o índice desses elementos em seus respectivos clusters
    for j in range(len(data)):
      row=data[j]
      best_centroid = 0 
      min_distance = np.inf
      
      for i in range(k):
        #Calcula a distância do elemento em relação ao centróide desse cluster
        d = euclidiana(centroids[i],row) 
         
        #Se ele está mais perto que a menor distância atual, esse centróide passa a ser o melhor candidato 
        if d < min_distance:
          min_distance = d
          best_centroid = i

      clusters[best_centroid].append(j)


    #Gera novos centróides a partir da média
    for ix, cluster in enumerate(clusters):
      
      averages = np.zeros(len(data[0])) # retorna um vetor de x zeros, tal que x é o número de colunas do dataset
     
      #Somatória dos elementos de cada coluna
      for col in range(len(data[0])):
        for i in cluster:
          averages[col] += data[i][col]
          
      
      #Dividindo a soma pela quantidade de elementos no cluster
      for j in range(len(averages)):
        averages[j] /= len(cluster)

      #Elegendo o novo centróide
      centroids[ix] = averages
  
  return clusters, centroids


def generateMatrixWithClusterColumn(clusters, dataset, classes):
  newMatrixWithCluster = [[] for i in range(len(dataset))]
  
  for cluster_idx,cluster in enumerate(clusters):
    for item_idx,item in enumerate(cluster):
      newMatrixWithCluster[item] = np.append(dataset[item], [cluster_idx])
  
  newMatrix = pd.DataFrame(newMatrixWithCluster)
  newMatrix['classe'] = classes
  return newMatrix

In [ ]:
# CELULA 4

#Calcula a silhueta simplificada, olhando apenas o centroide
def calculate_silhouette(kmeans, df):
  swc_sum = 0
  for i in range(df.shape[0]):
    dist_centroids = []
    for j in kmeans.cluster_centers_:
      dist_centroids.append(euclidiana(df[i], j))

    sorted(dist_centroids)
    a = dist_centroids[0]
    b = dist_centroids[1]
    s = (b-a)/max(a,b)
    swc_sum += s

  swc = swc_sum/df.shape[0]

  return swc

In [ ]:

# CELULA 5

def main(import_file_path, export_file_path, number_of_clusters):
  dataset, classe = le_arq(import_file_path)

  # abaixo, será retornado uma lista de duas dimensões com os índices de cada cluster
  clusters, centroids = Kcluster(dataset,number_of_clusters)

  kmeans = KMeans(n_clusters=number_of_clusters).fit(dataset)

  # newMatrix vai conter o dataset original com a nova coluna do cluster
  newMatrix = generateMatrixWithClusterColumn(clusters, dataset, classe)

  #class_table = createClassTable(newMatrix, number_of_clusters)
  sswc = calculate_silhouette(kmeans, dataset)
  print(sswc)
  
  # Insere o grupo correspondente na última coluna
  exportToCsv(newMatrix, export_file_path, str(sswc), str(number_of_clusters))

  print("=================================================================================================")
  print("Dataset: ", import_file_path)
  print("Aplicou o algorítmo k-means com k=", number_of_clusters)
  print("O dataset com a última coluna contendo o cluster do objeto pode ser encontrado em: ", export_file_path)
  print("=================================================================================================")

In [ ]:
#CELULA 6
# main()
main("/content/gdrive/My Drive/Agrupamentos/Atividade16/iris.data", "/content/gdrive/My Drive/Agrupamentos/Atividade16/", 2)
main("/content/gdrive/My Drive/Agrupamentos/Atividade16/iris.data", "/content/gdrive/My Drive/Agrupamentos/Atividade16/", 3)
main("/content/gdrive/My Drive/Agrupamentos/Atividade16/iris.data", "/content/gdrive/My Drive/Agrupamentos/Atividade16/", 4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


0.17958671250983416
Dataset:  /content/gdrive/My Drive/Agrupamentos/Atividade16/iris.data
Aplicou o algorítmo k-means com k= 2
O dataset com a última coluna contendo o cluster do objeto pode ser encontrado em:  /content/gdrive/My Drive/Agrupamentos/Atividade16/
-0.19446022799498697
Dataset:  /content/gdrive/My Drive/Agrupamentos/Atividade16/iris.data
Aplicou o algorítmo k-means com k= 3
O dataset com a última coluna contendo o cluster do objeto pode ser encontrado em:  /content/gdrive/My Drive/Agrupamentos/Atividade16/


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


0.0631499036237
Dataset:  /content/gdrive/My Drive/Agrupamentos/Atividade16/iris.data
Aplicou o algorítmo k-means com k= 4
O dataset com a última coluna contendo o cluster do objeto pode ser encontrado em:  /content/gdrive/My Drive/Agrupamentos/Atividade16/
